#       Shelter Animal Outcomes 4

## Logistic Regression

In [1]:
from sklearn.linear_model import LogisticRegression
import pandas as pd

In [2]:
df_train = pd.read_csv('../Shelter_train.csv')
df_test = pd.read_csv('../Shelter_test.csv')

In [3]:
x = df_train.ix[:, :-1]
y = df_train.ix[:, -1]
df_test = df_test.drop('ID', 1)

In [4]:
clf = LogisticRegression()
clf = clf.fit(x, y)
predictions = clf.predict_proba(df_test)
output = pd.DataFrame(predictions, columns=['Adoption', 'Died', 'Euthanasia', 'Return_to_owner', 'Transfer'])
output.index.names = ['ID']
output.index += 1
output.head()

,Adoption,Died,Euthanasia,Return_to_owner,Transfer
1,0.117482,0.008172,0.050876,0.236215,0.587255
2,0.437364,0.003482,0.030615,0.291800,0.236740
3,0.742515,0.004530,0.013035,0.059870,0.180050
4,0.301498,0.005703,0.032913,0.236961,0.422925
5,0.602574,0.002010,0.018305,0.261938,0.115174


In [5]:
output.to_csv('submission-logisticRegression.1.0.csv', index_label = 'ID')